In [1]:
import os
#os.environ["ASE_ABINIT_COMMAND"]="mpirun -np 8 abinit  < PREFIX.files > PREFIX.log"
os.environ["ABINIT_PP_PATH"]="/usr/share/abinit/psp:/usr/share/abinit/psp/HGH:/home/jochym/Projects/abinitio-methods/psp"

In [3]:
%pylab inline
import ase
from ase.build import bulk
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.calculators.abinit import Abinit
import ase.io
from ase import units as un
from ase.spacegroup import crystal

# Patch bug in ASE AbInit interface
import ase_patch
ase.io.abinit.read_abinit_out = ase_patch.read_abinit_out

from elastic import get_pressure, BMEOS, get_strain
from elastic import get_elementary_deformations, scan_volumes
from elastic import get_BM_EOS, get_elastic_tensor

from ipywidgets import interact
import nglview as nv

Populating the interactive namespace from numpy and matplotlib


In [10]:
def show_crystal(cryst):
    '''
    This function assumes cubic crystal on input. 
    If you want to use it for other structures you will need to adapt it.
    '''
    v = nv.NGLWidget()
    uc_a=1.5
    
    #cubic cell
    uc = ase.Atoms("C8", scaled_positions=[[0,0,0], 
                     [1,0,0],
                     [0,1,0],
                     [0,0,1],
                     [0,1,1],
                     [1,0,1],
                     [1,1,0],
                     [1,1,1],
                   ], cell=[uc_a,uc_a,uc_a])

    #cubic crystal
    a = cryst.cell.cellpar()[0]
    v.camera = 'orthographic'
    v.parameters = { "clipDist": 0 }

    o = v.add_structure(nv.ASEStructure(cryst))
    o.remove_ball_and_stick()
    o.add_spacefill(radius_scale=0.5, radiusType='covalent',
                            color_scheme='element', color_scale='rainbow')

    u = v.add_structure(nv.ASEStructure(uc))
    u.remove_spacefill()
    u.remove_ball_and_stick()
    u.add_line(cross_size=0.1)
    u.set_position(tuple((a-uc_a)*ones(3)/2))
    u.set_scale(a/uc_a)
    v.control.zoom(-1)
    v.control.spin([0,0,1], pi/2)
    v.control.center([a/2,a/2,a/2])
    return v, u, o, uc

In [11]:
a = 4.194
cryst = crystal(['Mg', 'O'], 
                [(0, 0, 0), (0.5, 0.5, 0.5)], 
                spacegroup=225,
                cellpar=[a, a, a, 90, 90, 90])

In [12]:
v, ucv, crv, uc = show_crystal(cryst)
v

NGLWidget()

In [14]:
@interact
def rot(a=(0.5,1.5,0.01), b=(-1,1,0.01)):
    m = array([[a,b,0],
               [0,1,0],
               [0,0,1]])
    cr = ase.Atoms(uc)
    cr.set_cell(dot(m, uc.cell.array), scale_atoms=True)
    ucv.set_coordinates(cr.get_positions())
    cr = ase.Atoms(cryst)
    cr.set_cell(dot(m, cryst.cell.array), scale_atoms=True)
    crv.set_coordinates(cr.get_positions())

interactive(children=(FloatSlider(value=1.0, description='a', max=1.5, min=0.5, step=0.01), FloatSlider(value=…

In [ ]:
# Create the calculator running on one, eight-core node.
# This is specific to the setup on my cluster.
# You have to adapt this part to your environment
calc = ClusterVasp(nodes=1, ppn=8)

# Assign the calculator to the crystal
cryst.set_calculator(calc)

# Set the calculation parameters
calc.set(prec = 'Accurate', xc = 'PBE', lreal = False,  
            nsw=30, ediff=1e-8, ibrion=2, kpts=[3,3,3])

# Set the calculation mode first.
# Full structure optimization in this case.
# Not all calculators have this type of internal minimizer!
calc.set(isif=3)


In [19]:
calc = Abinit(label='MgO', directory="work_07", v8_legacy_format=True,
              ecut= 200,
              pps = 'pawxml', pawecutdg = 800, xc = 'GGA',
              toldfe=1.0e-6,
              diemac=12,
              nstep=10,
              kptopt=1,
              ngkpt=[2,2,2],
              nshiftk=1,
              shiftk=[[0.5, 0.5, 0.5]],
              )

cryst.set_calculator(calc)
print(cryst.get_potential_energy(), cryst.get_stress()/un.GPa)

-8631.456471106283 [330.9648003 330.9648003 330.9648003   0.          0.          0.       ]


In [18]:
def scan_param(cryst, param, start=0, stop=1, steps=7, 
               lista=None, log_scale=True, int_par=False):
    '''
    Scan a range of parameter param and plot 
    '''
    fi = lambda x: x
    fo = lambda x: x

    if lista is None:
        if log_scale :
            fi = log
            fo = exp
        lista = linspace(fi(start), fi(stop), steps)
    else :
        int_par=False
    
    dat = []
    vs = []
        
    for v in lista:
        print(f'{param}={fo(v)}: ', end='')
        if int_par:
            cryst.calc.set(**{param: int(fo(v))})
        else :
            cryst.calc.set(**{param: fo(v)})
        vs.append(fo(v))
        dat.append([cryst.get_potential_energy(),
                    cryst.get_stress()[:3].mean()])
        print(f'{dat[-1][0]:.3f} eV ; {dat[-1][1]/un.GPa:6.3f} GPa',)
    dat = array(dat).T
    return vs, dat

In [ ]:
ecuts, eres = scan_param(cryst, 'ecut', 400, 700)

ecut=399.9999999999999: 

In [ ]:
figsize(6,6)
subplot(211)
plot(ecuts, eres[0], 'o')
ylabel('Energy (eV)')
subplot(212)
plot(ecuts, eres[1]/un.GPa, 'o')
ylabel('Pressure (GPa)')
xlabel('ecut (eV)');

In [ ]:
print("Residual pressure: %.3f bar" % (get_pressure(cryst.get_stress())))

In [ ]:
# Lets extract optimized lattice constant.
# MgO is cubic so a is a first diagonal element of lattice matrix
a=cryst.get_cell()[0,0]

# Clean up the directory
calc.clean()

systems=[]
# Iterate over lattice constant in the +/-5% range
for av in numpy.linspace(a*0.95,a*1.05,5):
    systems.append(crystal(['Mg', 'O'], [(0, 0, 0), (0.5, 0.5, 0.5)], 
                    spacegroup=225, cellpar=[av, av, av, 90, 90, 90]))

# Define the template calculator for this run
# We can use the calc from above. It is only used as a template.
# Just change the params to fix the cell volume
calc.set(isif=2)

# Run the calculation for all systems in sys in parallel
# The result will be returned as list of systems res
res=ParCalculate(systems,calc)

# Collect the results
v=[]
p=[]
for s in res :
    v.append(s.get_volume())
    p.append(get_pressure(s.get_stress()))

# Plot the result (you need matplotlib for this
plt.plot(v,p,'o')
plt.show()


$$
P(V)= \frac{B_0}{B'_0}\left[ \left({\frac{V}{V_0}}\right)^{-B'_0} - 1 \right]
$$

In [ ]:
# Switch to cell shape+IDOF optimizer
calc.set(isif=4)

# Calculate few volumes and fit B-M EOS to the result
# Use +/-3% volume deformation and 5 data points
deform=scan_volumes(cryst, n=5,lo=0.97,hi=1.03)

# Run the calculations - here with Cluster VASP
res=ParCalculate(deform,calc)

# Post-process the results
fit=get_BM_EOS(cryst, systems=res)

# Get the P(V) data points just calculated
pv=numpy.array(cryst.pv)

# Sort data on the first column (V)
pv=pv[pv[:, 0].argsort()]

# Print just fitted parameters
print("V0=%.3f A^3 ; B0=%.2f GPa ; B0'=%.3f ; a0=%.5f A" % ( 
        fit[0], fit[1]/units.GPa, fit[2], pow(fit[0],1./3)))

v0=fit[0]

# B-M EOS for plotting
fitfunc = lambda p, x: numpy.array([BMEOS(xv,p[0],p[1],p[2]) for xv in x])

# Ranges - the ordering in pv is not guarateed at all!
# In fact it may be purely random.
x=numpy.array([min(pv[:,0]),max(pv[:,0])])
y=numpy.array([min(pv[:,1]),max(pv[:,1])])


# Plot the P(V) curves and points for the crystal
# Plot the points
plt.plot(pv[:,0]/v0,pv[:,1]/units.GPa,'o')

# Mark the center P=0 V=V0
plt.axvline(1,ls='--')
plt.axhline(0,ls='--')

# Plot the fitted B-M EOS through the points
xa=numpy.linspace(x[0],x[-1],20)
plt.plot(xa/v0,fitfunc(fit,xa)/units.GPa,'-')
plt.title('MgO pressure vs. volume')
plt.xlabel('$V/V_0$')
plt.ylabel('P (GPa)')
plt.show()


$$
V_0 = 73.75 \text{ A}^3 \quad
B_0 = 170 \text{ GPa}  \quad
B'_0 = 4.3  \quad
a_0 = 4.1936 \text{ A}
$$

In [ ]:
# Switch to IDOF optimizer
calc.set(isif=2)

# Create elementary deformations
systems = get_elementary_deformations(cryst, n=5, d=0.33)

# Run the stress calculations on deformed cells
res = ParCalculate(systems, calc)

# Elastic tensor by internal routine
Cij, Bij = get_elastic_tensor(cryst, systems=res)
print("Cij (GPa):", Cij/units.GPa)


Library usage
=============

Simple Parallel Calculation
---------------------------

Once you have everything installed and running you can run your first
real calculation. The first step is to import the modules to your
program (the examples here use VASP calculator)

```python
from ase.spacegroup import crystal
import ase.units as units
import numpy
import matplotlib.pyplot as plt

from parcalc import ClusterVasp, ParCalculate

from elastic import get_pressure, BMEOS, get_strain
from elastic import get_elementary_deformations, scan_volumes
from elastic import get_BM_EOS, get_elastic_tensor
```

next we need to create our example MgO crystal:

``` {.sourceCode .python}
a = 4.194
cryst = crystal(['Mg', 'O'], 
                [(0, 0, 0), (0.5, 0.5, 0.5)], 
                spacegroup=225,
                cellpar=[a, a, a, 90, 90, 90])
```

We need a calculator for our job, here we use VASP and ClusterVasp
defined in the parcalc module. You can probably replace this calculator
by any other ASE calculator but this was not tested yet. Thus let us
define the calculator:

``` {.sourceCode .python}
# Create the calculator running on one, eight-core node.
# This is specific to the setup on my cluster.
# You have to adapt this part to your environment
calc = ClusterVasp(nodes=1, ppn=8)

# Assign the calculator to the crystal
cryst.set_calculator(calc)

# Set the calculation parameters
calc.set(prec = 'Accurate', xc = 'PBE', lreal = False,  
            nsw=30, ediff=1e-8, ibrion=2, kpts=[3,3,3])

# Set the calculation mode first.
# Full structure optimization in this case.
# Not all calculators have this type of internal minimizer!
calc.set(isif=3)
```

Finally, run our first calculation. Obtain relaxed structure and
residual pressure after optimization:

``` {.sourceCode .python}
print("Residual pressure: %.3f bar" % (
        get_pressure(cryst.get_stress())))
```

::: {.parsed-literal}
Residual pressure: 0.000 bar
:::

If this returns proper pressure (close to zero) we can use the obtained
structure for further calculations. For example we can scan the volume
axis to obtain points for equation of state fitting. This will
demonstrate the ability to run several calculations in parallel - if you
have a cluster of machines at your disposal this will speed up the
calculation considerably.

``` {.sourceCode .python}
# Lets extract optimized lattice constant.
# MgO is cubic so a is a first diagonal element of lattice matrix
a=cryst.get_cell()[0,0]

# Clean up the directory
calc.clean()

systems=[]
# Iterate over lattice constant in the +/-5% range
for av in numpy.linspace(a*0.95,a*1.05,5):
    systems.append(crystal(['Mg', 'O'], [(0, 0, 0), (0.5, 0.5, 0.5)], 
                    spacegroup=225, cellpar=[av, av, av, 90, 90, 90]))

# Define the template calculator for this run
# We can use the calc from above. It is only used as a template.
# Just change the params to fix the cell volume
calc.set(isif=2)

# Run the calculation for all systems in sys in parallel
# The result will be returned as list of systems res
res=ParCalculate(systems,calc)

# Collect the results
v=[]
p=[]
for s in res :
    v.append(s.get_volume())
    p.append(get_pressure(s.get_stress()))

# Plot the result (you need matplotlib for this
plt.plot(v,p,'o')
plt.show()
```

::: {.parsed-literal}
Workers started: 5
:::

![image](lib-usage_files/lib-usage_9_1.png)

Birch-Murnaghan Equation of State
---------------------------------

Let us now use the tools provided by the modules to calculate equation
of state for the crystal and verify it by plotting the data points
against fitted EOS curve. The EOS used by the module is a well
established Birch-Murnaghan formula (P - pressure, V - volume, B
-parameters):

$$P(V)= \frac{B_0}{B'_0}\left[
\left({\frac{V}{V_0}}\right)^{-B'_0} - 1
\right]$$

Now we repeat the setup and optimization procedure from the example 1
above but using a new Crystal class (see above we skip this part for
brevity). Then comes a new part (IDOF - Internal Degrees of Freedom):

``` {.sourceCode .python}
# Switch to cell shape+IDOF optimizer
calc.set(isif=4)

# Calculate few volumes and fit B-M EOS to the result
# Use +/-3% volume deformation and 5 data points
deform=scan_volumes(cryst, n=5,lo=0.97,hi=1.03)

# Run the calculations - here with Cluster VASP
res=ParCalculate(deform,calc)

# Post-process the results
fit=get_BM_EOS(cryst, systems=res)

# Get the P(V) data points just calculated
pv=numpy.array(cryst.pv)

# Sort data on the first column (V)
pv=pv[pv[:, 0].argsort()]

# Print just fitted parameters
print("V0=%.3f A^3 ; B0=%.2f GPa ; B0'=%.3f ; a0=%.5f A" % ( 
        fit[0], fit[1]/units.GPa, fit[2], pow(fit[0],1./3)))

v0=fit[0]

# B-M EOS for plotting
fitfunc = lambda p, x: numpy.array([BMEOS(xv,p[0],p[1],p[2]) for xv in x])

# Ranges - the ordering in pv is not guarateed at all!
# In fact it may be purely random.
x=numpy.array([min(pv[:,0]),max(pv[:,0])])
y=numpy.array([min(pv[:,1]),max(pv[:,1])])


# Plot the P(V) curves and points for the crystal
# Plot the points
plt.plot(pv[:,0]/v0,pv[:,1]/units.GPa,'o')

# Mark the center P=0 V=V0
plt.axvline(1,ls='--')
plt.axhline(0,ls='--')

# Plot the fitted B-M EOS through the points
xa=numpy.linspace(x[0],x[-1],20)
plt.plot(xa/v0,fitfunc(fit,xa)/units.GPa,'-')
plt.title('MgO pressure vs. volume')
plt.xlabel('$V/V_0$')
plt.ylabel('P (GPa)')
plt.show()
```

::: {.parsed-literal}
Workers started: 5 V0=74.233 A\^3 ; B0=168.19 GPa ; B0\'=4.270 ;
a0=4.20275 A
:::

![image](lib-usage_files/lib-usage_12_1.png)

If you set up everything correctly you should obtain fitted parameters
printed out in the output close to:

$$V_0 = 73.75 \text{ A}^3 \quad
B_0 = 170 \text{ GPa}  \quad
B'_0 = 4.3  \quad
a_0 = 4.1936 \text{ A}$$

Calculation of the elastic tensor
---------------------------------

Finally let us calculate an elastic tensor for the same simple cubic
crystal - magnesium oxide (MgO). For this we need to create the crystal
and optimize its structure (see :ref:`parcalc` above). Once we have an
optimized structure we can switch the calculator to internal degrees of
freedom optimization (IDOF) and calculate the elastic tensor:

``` {.sourceCode .python}
# Switch to IDOF optimizer
calc.set(isif=2)

# Create elementary deformations
systems = get_elementary_deformations(cryst, n=5, d=0.33)

# Run the stress calculations on deformed cells
res = ParCalculate(systems, calc)

# Elastic tensor by internal routine
Cij, Bij = get_elastic_tensor(cryst, systems=res)
print("Cij (GPa):", Cij/units.GPa)
```

::: {.parsed-literal}
Workers started: 10 Cij (GPa): \[ 338.46921273 103.64272667 152.2150523
\]
:::

To make sure we are getting the correct answer let us make the
calculation for $C_{11}, C_{12}$ by hand. We will deform the cell along
a (x) axis by +/-0.2% and fit the $3^{rd}$ order polynomial to the
stress-strain data. The linear component of the fit is the element of
the elastic tensor:

``` {.sourceCode .python}
from elastic.elastic import get_cart_deformed_cell

# Create 10 deformation points on the a axis
systems = []
for d in numpy.linspace(-0.2,0.2,10):
    systems.append(get_cart_deformed_cell(cryst, axis=0, size=d))

# Calculate the systems and collect the stress tensor for each system
r = ParCalculate(systems, cryst.calc)
ss=[]
for s in r:
    ss.append([get_strain(s, cryst), s.get_stress()])

ss=numpy.array(ss)
lo=min(ss[:,0,0])
hi=max(ss[:,0,0])
mi=(lo+hi)/2
wi=(hi-lo)/2
xa=numpy.linspace(mi-1.1*wi,mi+1.1*wi, 50)
```

::: {.parsed-literal}
Workers started: 10
:::

``` {.sourceCode .python}
# Make a plot
plt.plot(ss[:,0,0],ss[:,1,0]/units.GPa,'.')
plt.plot(ss[:,0,0],ss[:,1,1]/units.GPa,'.')

plt.axvline(0,ls='--')
plt.axhline(0,ls='--')

# Now fit the polynomials to the data to get elastic constants
# C11 component
f=numpy.polyfit(ss[:,0,0],ss[:,1,0],3)
c11=f[-2]/units.GPa

# Plot the fitted function
plt.plot(xa,numpy.polyval(f,xa)/units.GPa,'-', label='$C_{11}$')

# C12 component
f=numpy.polyfit(ss[:,0,0],ss[:,1,1],3)
c12=f[-2]/units.GPa

# Plot the fitted function
plt.plot(xa,numpy.polyval(f,xa)/units.GPa,'-', label='$C_{12}$')
plt.xlabel('Relative strain')
plt.ylabel('Stress componnent (GPa)')
plt.title('MgO, strain-stress relation ($C_{11}, C_{12}$)')
plt.legend(loc='best')
# Here are the results. They should agree with the results
# of the internal routine.
print('C11 = %.3f GPa, C12 = %.3f GPa => K= %.3f GPa' % (
        c11, c12, (c11+2*c12)/3))

plt.show()
```

::: {.parsed-literal}
C11 = 325.005 GPa, C12 = 102.441 GPa =\> K= 176.629 GPa
:::

![image](lib-usage_files/lib-usage_18_1.png)

If you set up everything correctly you should obtain fitted parameters
printed out in the output close to:

    Cij (GPa): [ 340   100   180]

With the following result of fitting:

    C11 = 325 GPa, C12 = 100 GPa => K= 180 GPa

The actual numbers depend on the details of the calculations setup but
should be fairly close to the above results.
